# We use the porfolio method : find the best bag size for each instance 

In [2]:
import pandas as pd
import numpy as np 
knapsack = pd.read_csv('Knapsack_Instances.csv',header=None,skip_blank_lines=False)
knapsack = knapsack.fillna(0)
knapsack.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,98.00,113.00,108.00,66.00,95.00,91.00,35.00,49.00,27.00,91.00,...,16.00,20.00,67.00,77.00,9.00,26.00,3.00,117.00,60.00,49.00
1,66.61,190.66,134.96,67.32,65.48,27.08,69.65,17.23,44.05,34.92,...,11.78,20.46,79.58,123.30,6.35,21.88,4.79,119.31,23.00,35.33
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,76.00,105.00,117.00,4.00,107.00,109.00,66.00,77.00,112.00,6.00,...,34.00,58.00,122.00,33.00,7.00,23.00,76.00,18.00,93.00,93.00
4,120.69,58.57,67.50,3.61,145.38,83.47,47.31,101.94,116.52,3.89,...,10.07,24.08,157.83,36.92,7.00,18.11,102.58,24.18,98.03,129.91


In [9]:
#Calculate the maximum value of the per instance
def bag(n,c,w,p):
    res=[[-1 for j in range(c+1)]for i in range(n+1)]
    for j in range(c+1):
        res[0][j]=0
    for i in range(1,n+1):
        for j in range(1,c+1):
            res[i][j]=res[i-1][j]
            if(j>=w[i-1]) and res[i-1][j-w[i-1]]+p[i-1]>res[i][j]:
                res[i][j]=res[i-1][j-w[i-1]]+p[i-1]
    return res

In [12]:
#Get the list of "choose or not line". 
def show(n,c,w,res):
    choose = []
    x=[False for i in range(n)]  
    j=c  
    for i in range(n,0,-1):  
        if res[i][j]>res[i-1][j]:  
            x[i-1]=True
            j = j - w[i-1]   
    for i in range(n):  
        if x[i]:  
            choose+=[1]
        else:
            choose+=[0]
    return choose

In [17]:
# each instance has 20 possible bagsize cause generally we can estimate the bagsize will below 120. 
# each bagsize will run through 100 uniform distributed bagsize. 
best_bags = []
for i in range(0,1500,3):#run through 500 instances, i = 0,3,6,....1497 i/3 = 0,1,2....499
    space = list(knapsack.loc[i][0:])
    space = [int (i) for i in space]
    reward = list(knapsack.loc[i+1][0:])
    true_list = list(knapsack.loc[i+2][0:])
    total_space_list = np.random.uniform(100,150,100)#Generate 100 teacher bags. 
    bags ={}
    for bag_size in range(100,120):#列举20个 bagsize
        reward_sum = 0 
        for j in range(100):#老师会给的100个 bagsize 
            if bag_size <= total_space_list[j]:
                matrix = bag(50,bag_size,space,reward) # 现在 matrix 是 51行，101列
                reward_sum = reward_sum + matrix[-1][-1]#每一个 bagsize 的最大值
        bags[bag_size]=reward_sum/100
    best_bag = max(bags, key=bags.get) #这个 instance 最好的 bagsize
    best_bags += [best_bag] # 500个 instance 最好的 bagsizes     

In [33]:
import csv
with open("abcd.csv","w") as csvfile:
        writer = csv.writer(csvfile)
        for i in range(0,1500,3):
            # The bag size number should be : i/3
            space = list(knapsack.loc[i][0:])
            space = [int (i) for i in space] # first row of an instance
            reward = list(knapsack.loc[i+1][0:])
            matrix = bag(50,best_bags[int(i/3)],space,reward)
            a = show(50,best_bags[int(i/3)],space,matrix)
            writer.writerow(space)
            writer.writerow(reward)
            writer.writerow(a)
